# Data Processing Pipeline for *"Bundling Measures for Food Systems Transformation: a global, multimodel assessment"*
This outlines the data processing for ***"Bundling measures for food systems transformation: a global multimodel assessment"***, by Sundiang et al, 2025, *The Lancet Planetary Health*

This uses the custom package AgMIP Processing PipeLinE or `applepy`. Note that `applepy` is still in development. The version used here may not be the most current release. Therefore it is important to use the included `applepy` folder in this repository. 

```python
import sys
import pandas as pd
import json
import pickle
import os
from os.path import join as pjoin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import logging
from multiprocessing import Pool
import csv

sys.path.append("..")
import applepy as apy
from applepy.utils.calculations.basic import *
from applepy.utils.calculations.bias_correction import *
from applepy.utils.calculations.emissions import *
from applepy.utils.calculations.land import *

from applepy.utils.preprocessing.merge import *
from applepy.utils.preprocessing.checks import *
from applepy.pipeline.pipeline import *
from applepy.visualization.coverage_map import *

from applepy.utils.helper import * 
```

## Step 0: Project Setup
0.1 Update the AgMIP template with relevant indicators to the specific project

## Step 1: myGeoHub
The first step of the processing is done through the AgMIP GlobalEcon Data Submission tool (Zhao et al, 2023), hosted through MyGeoHub.

1. Each AgMIP project should be setup with its own `RulesTables.xlsx` file. The `RulesTables` file is a lookup table for the submission tool to validate the submission's indicators, units, scenario names, etc. to make sure that it follows the AgMIP reporting convention.
This `RulesTables` file has the following sheets
* 'Version'
* 'ModelTable'
* 'ScenarioTable'
* 'RegionFixTable'
* 'VariableTable'
* 'VariableUnitValueTable'
* 'ItemTable'
* 'UnitTable'
* 'YearTable'
* 'ValueFixTable'
* 'RegionTable

2. A validation step in the submission tool allows the user to override or replace values that do not match the `RulesTables`. Here one can choose among the allowed values, and the submission tool will replace the invalid values. If there is no corresponding valid value for the item that caused the exception, one can select to override the exception. Submissions with exceptions are moved to the pending folder and needs manual review (which is discussed in the next step)

3. The pending submissions will have two files: the original file and another overrides file (`*_OVERRIDES.csv`)

**References:**
 Lan Zhao; Jaewoo Shin; Rob Campbell; Raziq Ramli (2023), "AgMIP GlobalEcon Data Submission," https://mygeohub.org/resources/agmipup. (DOI: 10.21981/ZTHG-RK49).

## Step 2: Setting up applepy


**0. Install the python environment file:**

`conda env create -n el-modelling_v3 -f el-modelling_v3.yml`

```python
import sys
sys.path.append("..")

import applepy as apy
from applepy.utils.calculations.basic import *
from applepy.utils.calculations.bias_correction import *
from applepy.utils.calculations.emissions import *
from applepy.utils.calculations.land import *

from applepy.utils.preprocessing.merge import *
from applepy.utils.preprocessing.checks import *
from applepy.pipeline.pipeline import *
from applepy.visualization.coverage_map import *

from applepy.utils.helper import * 
```

**1. Generating the `*_OVERRIDES_fix` file.** 

The original file that comes from myGeoHub has the following structure (without the headers):
<center>

| override | column | closest value |
|----------|--------|--------|
| YILD_endoTC | Variable | YILD |
| Nr/Nr | Unit | TgN/year |
| unitless | Unit | index |
| t dm/ha | Unit | t/ha|
 </center>

we replace the last column with a `status` column and generate an `*_OVERRIDES_fix.csv` file
<center>

| override | column | status |
|----------|--------|--------|
| YILD_endoTC | Variable | False |
| Nr/Nr | Unit | True |
| unitless | Unit | True |
| t dm/ha | Unit | dm t/ha|
</center>

The `status` column takes in a boolean value or a replacement value. A False value will remove the entries from the dataset (row 1 above), and True will keep the entries (rows 2 and 3). If the value is not boolean and is a string datatype, the value will be replaced by the string (row 4).

```python
data_dir = '../data/'
fps = sorted([pjoin(data_dir,x) for x in os.listdir(data_dir) if (x.endswith('OVERRIDES.csv'))])
col_names = ['override','column','status']

for fp in fps:
    override_df = pd.read_csv(fp, names=col_names)
    override_df['status'] = False
    save_fp = fp.split('/')[-1].split('.csv')[0]+'_fix.csv'
    override_df.to_csv(pjoin(data_dir,save_fp),index=False,header=False)

```

## Step 3: Running the EL2 Pipeline
### On a single submission ([source](../applepy/pipeline/pipeline.py#L16))
**0.** ([source](../applepy/utils/helper.py#L94)) The model csv file is read into a Pandas DataFrame. This step assumes that the model submission has already been passed through the AgMIP submission tool above (or has the same format, i.e. no headers, comma separated, and has the column order `['model','scenario','region','variable','item','unit','year','value']`). All outputs will be saved in the same folder as the file and will be named with the model name as the prefix.

**1. Checking duplicates.** ([source](../applepy/utils/preprocessing/checks.py#L4))
Duplicated entries that match all values in the column are dropped (only one copy of the entry is kept), while entries that have a different values (`scenario, region, variable, item, unit and year` are the same) are separated from the clean DataFrame into a `<model>_duplicates.csv` file. This file is saved in a new folder `<location of model csv file>/duplicates`.

**2. Setting aside variables to keep** 
This step looks at the `Keep` column in the `RulesTable.xlsx` sheet `'VariableUnitValueTable'`. All the variables that are marked `True` in this column are set aside and added back *after* steps 3 and 4

**3. Checking overrides** ([source](../applepy/utils/preprocessing/checks.py#L37))
This steps handles the overrides according to the `*_OVERRIDES_fix.csv` file. If this file does not exist, the entire DataFrame is kept as is. Values that are set to `False`, and the list of overridden entries are kept in `*_overrides-list.csv`. This file is saved in a new folder `<location of model csv file>/overrides`.Values that are set to `True` are kept in the DataFrame as is. And Values that are neither are replaced by the value in the `status` column. 

**4. Checking against template** ([source](../applepy/utils/preprocessing/checks.py#L78))
The DataFrame is checked against the `RulesTable.xlsx` sheet `'VariableUnitValueTable'`. This is the final check whether each variable has the correct expected unit. Exeptions are removed and moved to ``<location of model csv file>/template-checked/*_template-exceptions-list.csv'`

**5. Calculating percentage changes** ([source](../applepy/utils/calculations/bias_correction.p#L137))

* Get base year (for this project we used 2020). If the model does not report that base year, interpolate using values before and after the base year (the submission should have years reported ***before and after**)
* The DataFrame is grouped by `'model','variable','item','region','unit'` in order to calculate the percentage change from the base year, and the same years across scenarios. 
* Additional columns are created:
    * `'BAU_ref_year'`: base year
    * `'percent_change_BAU_ref_year'`: percentage change from base year
    * `'diff_BAU_ref_year'`: absolute difference between scenario and year from BAU base year
    * `'percent_change_BAU'`: percentage from BAU of the same year
    * `'diff_BAU'`: absolute difference between scenario and year from BAU of the same year
    * `'percent_change_ELM'`: percentage change from ELM of the same year
    * `'diff_ELM'`: absolute difference between scenario and year from ELM of the same year



```python
fp = '../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/GLOBIOM-full_250819.csv'
el2_pipeline(fp)
```

```
# example output of el2_pipeline(fp)

PROCESSING FILE : model
>> original DataFrame length: 745958


>> checking duplicates
Found 70 duplicated entries
...0 of them have conflicting values...
All files will be saved in: ../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/duplicates


>> setting aside variables to keep
... set aside variables to keep. DataFrame length: 694111, 93.0% of the original df
>> checking overrides
... no overrides file found!



>> checking against template
Template exceptions removed: 0
... template checked. DataFrame length: 669810, 90.0% of the original df
... concatenating template-checked DataFrame with the kept overrides and variables to keep...
>> checking duplicates again
Found 0 duplicated entries
...0 of them have conflicting values...
... DataFrame length: 721587, 97.0% of the original df
All files will be saved in: ../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/template-checked


>> calculating percentage changes
All files will be saved in: ../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff
Processing file: model_template-checked
All files will be saved in: ../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/logs

```

### On multiple submissions (multiprocessed) ([source](../applepy/pipeline/pipeline.py#L160))

**NOTE:** save submission files in a folder. `el2_pipeline_multiprocess` takes in a path to a folder

```python
data_dir = '../data/[folder of submissions]'
el2_pipeline_multiprocess(data_dir)
```

## Optional: Additional Emission and Land calculations

### Additional Emissions Calculations ([source](../applepy/utils/calculations/emissions.py))
 Some models are able to report more types of non-CO2 emissions and include these in their reporting of total emissions. To standardize the greenhouse gasses reported in the total non-CO2 emissions, we calculate the total methane and nitrous oxide emissions reported by the models. The outputs of this additional calculations are the variables (note this is only done for `AGR`):
 * `EMIS_added`: added values for CO2, CH4, and N2O
 * `EMIS_nonCO2`: added values for CH4 and N2O
 * `ECH4_share`: share of CH4 in `EMIS_added`
 * `ECO2_share`: share of CO2 in `EMIS_added`
 * `EN2O_share`: share of N2O in `EMIS_added`
 * `nonCO2_share`: share of non-CO2 greenhouse gases (CH4 and N2O) in `EMIS_added`

### Additional Land Calculations ([source](../applepy/utils/calculations/land.py))
 To standardize the reporting of agricultral land to include only cropland and grassland, we calculate additional variables that 
 :
 * `LAND_added`: includes items:
    * `AGR_added`: added values for cropland (`CRP`) and grassland (`GRS`)
    * `ONV_added`: added values for other natural vegetation (`ONV`) and forests (`FOR`), if they were reported
    * `LAND_tot`: `AGR_added` and `ONV_added`
    * `CRP`: original `CRP` value reported
    * `GRS`: original `GRS` reported
 * `LAND_share`: includes items:
    * `CRP_share`: share of `CRP` in `LAND_tot`
    * `GRS_share`: share of `GRS` in `LAND_tot`
    * `AGR_share`: share of `AGR_added` in `LAND_tot`
    * `ONV_share`: share of `ONV_added` in `LAND_tot`
    * `CRP_AGR_share`: share of `CRP` in `AGR_added`
    * `GRS_AGR_share`: share of `GRS` in `AGR_added`

**NOTE:** all of these calulations are done in using the absolute values that each model reports. Following the summations, the percentage changes are calulated the same as the core dataset


```python
fp = "../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/model_template-checked_pc-diff_interp-2020.csv"
run_emissions_calcs(fp)
run_land_calcs(fp)
```

```
## example output of additional emission and land calcs

All files will be saved in: ../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/emissions

Statistics on the difference between added emissions (CH4, N2O, and CO2) and total emisisons reported
          count          mean           std       min           25%  50%  \
model                                                                      
GLOBIOM  2052.0 -3.411326e-09  1.515930e-07 -0.000001 -5.684342e-14  0.0   

                  75%       max  
model                            
GLOBIOM  5.684342e-14  0.000001  

>> Saving wide DataFrame to ../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/emissions/model_template-checked_pc-diff_interp-2020_EMIS-calcs-w.csv
>> Saving long DataFrame to ../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/emissions/model_template-checked_pc-diff_interp-2020_EMIS-calcs.csv
>> Running percentage change calculations...
Processing file: model_template-checked_pc-diff_interp-2020_EMIS-calcs
All files will be saved in: ../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/emissions/logs
100%|██████████| 108/108 [00:24<00:00,  4.33it/s]
Done. Saving file to ../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/emissions/model_template-checked_pc-diff_interp-2020_EMIS-calcs_pc-diff_interp-2020.csv
All files will be saved in: ../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/land

>> Saving wide DataFrame to ../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/land/model_template-checked_pc-diff_interp-2020_LAND-calcs-w.csv
>> Saving long DataFrame to ../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/land/model_template-checked_pc-diff_interp-2020_LAND-calcs.csv
>> Running percentage change calculations...
Processing file: model_template-checked_pc-diff_interp-2020_LAND-calcs
All files will be saved in: ../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/land/logs
100%|██████████| 162/162 [00:38<00:00,  4.22it/s]
Done. Saving file to ../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/land/model_template-checked_pc-diff_interp-2020_LAND-calcs_pc-diff_interp-2020.csv
```

## Step 4: Merging and updating the dataset
1. Merging
2. Updating an                  existing dataset

```python
# using the function merge_fps([list of filepaths]) to merge datasets
fps = ["../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/model_template-checked_pc-diff_interp-2020.csv", # core submission file
       "../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/emissions/model_template-checked_pc-diff_interp-2020_EMIS-calcs_pc-diff_interp-2020.csv", # additional emissions calcs
        "../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff/land/model_template-checked_pc-diff_interp-2020_LAND-calcs_pc-diff_interp-2020.csv" # additional land calcs
       ] 
output_dir = '../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/pc-diff'
check_path(output_dir)
merge_fps(fps,save=True,output_dir=output_dir,merge_fn=f"flw-update_GLOBIOM_emis-land-calcs_{time.strftime('%y%m%d')}.csv")
```

```python
### Example for partial update: GLOBIOM's PARTIAL FLW UPDATE
fp_old = '../data/2025-06-13_FLW-sensitivity/merged_files/flw-update_emis-land-calcs_250731.csv'
df_old = pd.read_csv(fp_old,index_col=0)

fp_new = '../data/2025-06-13_FLW-sensitivity/250819_GLOBIOM-partial-update/GLOBIOM_FOUR_WASTE_SCENARIOS_ONLY_08182025_104106.csv'
df_new = AgMIP_read_raw_csv(fp_new)

models_to_replace = df_new.model.unique()
scenarios_to_replace = df_new.scenario.unique()

# Merge GLOBIOM's new scenarios into the old one
# so get all the entries for GLOBIOM, but not the ones for the scenarios that will be replaced
df_old = df_old[(df_old.model.isin(models_to_replace)) &
                (~df_old.scenario.isin(scenarios_to_replace))]

cols = ['model', 'scenario', 'region', 'variable', 'item', 'unit', 'year',
       'value']
df_update = pd.concat([df_new, df_old]).reset_index(drop=True)[cols]
        
df_update.to_csv("/".join(fp_new.split("/")[:-1])+f"/GLOBIOM-full_{time.strftime('%y%m%d')}.csv",index=False)
```

## Step 5 (Optional): Model Specific adjustments

### Deal with variables that report dry matter
IMAGE and CAPRI report dry matter as separate variables. In this step, the dry matter variables (`*_dm`) units are replaced to add `dm` (e.g. `1000 t` to `1000 t dm`). Then the suffix is removed and combined to the base variable (e.g. `PROD_dry` to `PROD`)

```python
fp = "../data/2025-06-13_FLW-sensitivity/pc-diff_all/merged/flw-update_250630.csv"
df = pd.read_csv(fp,index_col=0)

#replace CAPRI *_dry to *_dm
df['variable'] = df['variable'].str.replace(r'_dry$', '_dm', regex=True)

# load dm_unit lookup table
dm_units_fp = "../applepy/template/dm_units.json"
with open(dm_units_fp) as json_data:
    dm_units = json.load(json_data)
    json_data.close()

# get list of *_dm vars
dm_vars = df[df.variable.str.endswith("_dm")].variable.unique()

# # check current data units: 
# for k in dm_units.keys():
#     model_units = df[df.variable==k].unit.unique()
#     print(k,model_units,dm_units[k])

for v in dm_vars:
    # replace units with dm_unit
    df.loc[df.variable==v,'unit'] = dm_units[v]
    
    # remove '*_dm' suffix from variable
    df.loc[df.variable==v,'variable'] = v.split('_dm')[0]
    # print(v, model_units,df[df.variable==v].unit.unique(), dm_units[v])

# save df
df.to_csv(fp.split(".csv")[0]+'_dm-fixed.csv')
```

## Step 6: Decomposition analysis ([source](applepy/utils/calculations/decomposition.py))

```python
from applepy.utils.calculations.decomposition import *
```

```python
# drop ELM_MITI and replace EL2 with ELM_MITI since these two scenarios are the same
df = df[df.scenario!='ELM_MITI']
# rename EL2 to ELM_MITI
df.loc[df.scenario=='EL2','scenario'] = 'ELM_MITI'
scenarios = ['BAU',
             'BAU_PROD','BAU_WAST','BAU_DIET','BAU_MITI',
             'ELM',
             'ELM_PROD','ELM_WAST','ELM_DIET','ELM_MITI']

df = df[df.scenario.isin(scenarios)]
for model in df.model.unique():
    print(model, sorted(df[df.model==model].scenario.unique()))

variables = df.variable.unique()
items = df.item.unique()
regions = ['CAN','USA','BRA','OSA','FSU','EUR','MEN','SSA','CHN','IND','SEA','OAS','ANZ','WLD']
years = [2050.0]
df = df[(df.variable.isin(variables)) &
        (df.item.isin(items))         &
        (df.region.isin(regions))     &
        (df.year.isin(years))         ]
grouped = df.groupby(['model','region','variable','item','unit','year'])

drivers = ['DIET','PROD','MITI','WAST']
dc_dict = []
values = ['value','percent_change_BAU','percent_change_BAU_ref_year'] 
for i, (model,region,variable,item,unit,year) in enumerate(tqdm(grouped.groups.keys())):
    k_df = grouped.get_group((model,region,variable,item,unit,year))
    for driver in drivers:
        for value in values: 
            for normalized in [True,False]:
                try:
                    dc_dict.append(decompose_driver_effect_filtered(k_df,value,driver,normalized=normalized, use_pandas=True, full_dict=True))
                except Exception as e:
                    print(model,region,variable,item,unit,year,e)
dc_df = pd.DataFrame.from_dict(dc_dict)
cols = ['individual', 'total', 'interaction', 'ELM','EL2', 'BAU', 'ELM_driver', 'BAU_driver']
for col in cols:
    dc_df[col] = [np.nan if (x.size==0) or (np.isnan(x[0])) else x[0] for x in dc_df[col].values]
dc_df.to_csv(pjoin(output_dir,base_filename+'_decomposed.csv'),index=False)
```

```python
# convert to long format
dc_df_l = dc_df.melt(id_vars=['model','region','variable','item','unit','year','driver','normalized','value_type'],value_vars=['individual','total','interaction'],var_name='effect')
# dc_df_l.value = [np.nan if (x.size==0) or (np.isnan(x[0])) else x[0] for x in dc_df_l.value.values]
dc_df_l.dropna(inplace=True)
dc_df_l.to_csv(pjoin(output_dir,base_filename+'_decomposed_l.csv'),index=False)
```